In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec, KeyedVectors, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [3]:
# Load the preprocessed data
train_df = pd.read_csv("/content/drive/MyDrive/NLPCoursework/Task2/preprocessed_training_data.csv")
test_df = pd.read_csv("/content/drive/MyDrive/NLPCoursework/Task2/preprocessed_testing_data.csv")

In [4]:
train_df.head()

,Text,Category
0,argentin 198687 grainoilse registr argentin gr...,wheat
1,champion product ch approv stock split champio...,earn
2,comput termin system cpml complet sale comput ...,acq
3,cobanco inc cbco year net shr 34 ct v 119 dlr ...,earn
4,ohio mattress omt may lower 1st qtr net ohio m...,acq


In [5]:
test_df["Category"].value_counts()

earn            1087
acq              699
crude            144
interest         112
money-fx         112
trade             88
ship              69
wheat             38
sugar             31
money-supply      30
Name: Category, dtype: int64

In [6]:
train_df["Category"].value_counts().head(6)

earn        2850
acq         1613
interest     312
crude        298
trade        289
money-fx     274
Name: Category, dtype: int64

In [7]:
top6_cat = ['earn', 'acq', 'interest', 'crude', 'trade', 'money-fx']

In [8]:
train_df = train_df[train_df['Category'].isin(top6_cat)]
test_df = test_df[test_df['Category'].isin(top6_cat)]

In [9]:
# split the dataset into training and validation datasets
train_x, valid_x, train_y, valid_y = train_test_split(train_df["Text"], train_df['Category'], test_size=0.2)

In [10]:
test_x = test_df['Text']

In [11]:
test_y = test_df['Category']

In [12]:
# label encode the target variable
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)
valid_y = encoder.fit_transform(valid_y)

TF IDF

In [13]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(train_df["Text"])

xtrain_tfidf =  tfidf_vect.transform(train_x)
xtest_tfidf =  tfidf_vect.transform(test_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

In [14]:
print("TF-IDF Vectors for Train Set:")
print(xtrain_tfidf[:5])

TF-IDF Vectors for Train Set:
  (0, 4983)	0.12466984970136008
  (0, 4970)	0.06769275632188845
  (0, 4746)	0.5530343138515957
  (0, 4617)	0.05313570678635878
  (0, 4570)	0.05599911546047879
  (0, 4537)	0.06011349559557074
  (0, 4083)	0.1112892846651573
  (0, 4068)	0.07829637616953583
  (0, 3989)	0.06923601917489916
  (0, 3890)	0.09128276836825949
  (0, 3766)	0.07133791530339943
  (0, 3738)	0.07916748070113569
  (0, 3720)	0.07270494824093417
  (0, 3696)	0.06896573969615549
  (0, 3485)	0.06619222014340748
  (0, 3480)	0.11984917145564804
  (0, 3419)	0.04659517435104641
  (0, 3349)	0.09807713274902893
  (0, 3305)	0.06951072966164036
  (0, 3298)	0.05135715914131634
  (0, 3184)	0.05734993886778597
  (0, 3160)	0.2019612816268285
  (0, 3159)	0.4258975538763477
  (0, 2712)	0.05672573293256799
  (0, 2634)	0.033255227997442884
  :	:
  (4, 3497)	0.07294570935551443
  (4, 3473)	0.06162831967970897
  (4, 3419)	0.044401735281147846
  (4, 3401)	0.053216359410228814
  (4, 3362)	0.6587507061361155
  (4, 

In [15]:
print("TF-IDF Vectors for Test Set:")
print(xtest_tfidf[:5])

TF-IDF Vectors for Test Set:
  (0, 4978)	0.025644970218568826
  (0, 4970)	0.04984878374926654
  (0, 4968)	0.0412930556433193
  (0, 4953)	0.05127504187863454
  (0, 4948)	0.04038852771247219
  (0, 4945)	0.05006717965839581
  (0, 4942)	0.02766466861698641
  (0, 4898)	0.036856144843223596
  (0, 4894)	0.036398906492761667
  (0, 4867)	0.02294098364496367
  (0, 4853)	0.03057167825333231
  (0, 4848)	0.03462393853125179
  (0, 4842)	0.027419938180078696
  (0, 4804)	0.0431772281236091
  (0, 4798)	0.06564297539836479
  (0, 4760)	0.0441283215150272
  (0, 4759)	0.04038852771247219
  (0, 4682)	0.3238100465120762
  (0, 4673)	0.036128469081934254
  (0, 4617)	0.31303205848052135
  (0, 4610)	0.04067555128894032
  (0, 4596)	0.04662999834969598
  (0, 4593)	0.020898246496556673
  (0, 4592)	0.0668520329125355
  (0, 4584)	0.023532355413731467
  :	:
  (4, 1890)	0.05367657304671917
  (4, 1771)	0.41352491418805815
  (4, 1696)	0.06018836543984334
  (4, 1545)	0.03423933746401258
  (4, 1195)	0.09217023178784779
  (

In [16]:
print("TF-IDF Vectors for Valid Set:")
print(xvalid_tfidf[:5])

TF-IDF Vectors for Valid Set:
  (0, 4722)	0.14091204274970187
  (0, 4537)	0.0847321220445157
  (0, 4491)	0.0851820316597906
  (0, 4168)	0.18596437720929757
  (0, 4113)	0.15356511419893928
  (0, 4070)	0.06363068501510123
  (0, 4068)	0.07357436169157817
  (0, 3753)	0.09530457409273707
  (0, 3708)	0.10745735885391856
  (0, 3704)	0.11908942720911796
  (0, 3658)	0.17098686241401662
  (0, 3184)	0.040418394998051006
  (0, 3031)	0.08859697386982437
  (0, 2791)	0.17572661206122592
  (0, 2634)	0.09374886651925612
  (0, 2504)	0.10236242145112188
  (0, 2203)	0.0895338490052771
  (0, 2045)	0.47806438217645164
  (0, 2027)	0.38182962368234125
  (0, 1971)	0.13878622826925696
  (0, 1899)	0.042448753309077214
  (0, 1867)	0.10789885844929245
  (0, 1525)	0.5449876302420936
  (0, 1499)	0.06690217805356079
  (0, 1309)	0.12353936290566873
  :	:
  (4, 3225)	0.0946214902981568
  (4, 3184)	0.024380319724873957
  (4, 3159)	0.14484435364417958
  (4, 2727)	0.06982459162631521
  (4, 2638)	0.08052168693633978
  (4, 

Count

In [17]:
# create a count vectorizer object
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(train_df['Text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xtest_count =  count_vect.transform(test_x)
xvalid_count =  count_vect.transform(valid_x)

In [18]:
print("Count Vectors for Train Set:")
print(xtrain_count[:5])

Count Vectors for Train Set:
  (0, 3455)	1
  (0, 5651)	1
  (0, 9472)	1
  (0, 9802)	2
  (0, 9803)	1
  (0, 9812)	1
  (0, 9964)	1
  (0, 10182)	1
  (0, 10358)	1
  (0, 10398)	1
  (0, 10584)	1
  (0, 10585)	1
  (0, 11501)	4
  (0, 11503)	1
  (0, 12183)	1
  (0, 12599)	2
  (0, 12672)	1
  (0, 12674)	1
  (0, 12749)	2
  (0, 12861)	2
  (0, 13550)	1
  (0, 13552)	1
  (0, 13640)	1
  (0, 13742)	1
  (0, 14060)	1
  :	:
  (4, 16720)	1
  (4, 17046)	1
  (4, 18122)	1
  (4, 18589)	1
  (4, 18836)	1
  (4, 19443)	1
  (4, 19700)	5
  (4, 19871)	1
  (4, 19962)	1
  (4, 20166)	1
  (4, 20270)	1
  (4, 21080)	1
  (4, 21190)	4
  (4, 21215)	1
  (4, 21602)	1
  (4, 21897)	1
  (4, 22307)	4
  (4, 22763)	3
  (4, 23681)	1
  (4, 23849)	1
  (4, 23953)	1
  (4, 24677)	1
  (4, 24723)	1
  (4, 25467)	2
  (4, 25839)	1


In [19]:
print("Count Vectors for Test Set:")
print(xtest_count[:5])

Count Vectors for Test Set:
  (0, 80)	1
  (0, 1674)	1
  (0, 1988)	1
  (0, 2573)	1
  (0, 4187)	1
  (0, 4188)	1
  (0, 6295)	1
  (0, 6675)	1
  (0, 7992)	1
  (0, 9408)	1
  (0, 9724)	1
  (0, 9770)	1
  (0, 9823)	1
  (0, 9887)	1
  (0, 10118)	2
  (0, 10142)	2
  (0, 10182)	4
  (0, 10238)	1
  (0, 10269)	2
  (0, 10309)	1
  (0, 10455)	1
  (0, 10577)	1
  (0, 10578)	2
  (0, 10583)	1
  (0, 10605)	1
  :	:
  (4, 14654)	1
  (4, 16732)	1
  (4, 16792)	1
  (4, 16969)	2
  (4, 17124)	1
  (4, 18265)	1
  (4, 18628)	1
  (4, 18863)	1
  (4, 18928)	19
  (4, 19255)	1
  (4, 20255)	1
  (4, 20773)	1
  (4, 21059)	2
  (4, 21135)	2
  (4, 21144)	4
  (4, 22056)	1
  (4, 22698)	1
  (4, 22881)	1
  (4, 23529)	2
  (4, 24003)	1
  (4, 24330)	1
  (4, 24354)	1
  (4, 24497)	2
  (4, 24704)	1
  (4, 25138)	13


In [20]:
print("Count Vectors for Valid Set:")
print(xvalid_count[:5])

Count Vectors for Valid Set:
  (0, 2578)	1
  (0, 3274)	1
  (0, 9802)	1
  (0, 9803)	1
  (0, 9964)	1
  (0, 10940)	1
  (0, 11735)	1
  (0, 12041)	2
  (0, 12460)	1
  (0, 12552)	3
  (0, 13075)	1
  (0, 13640)	1
  (0, 13742)	1
  (0, 14084)	1
  (0, 14335)	3
  (0, 14387)	3
  (0, 14933)	1
  (0, 16145)	1
  (0, 16720)	2
  (0, 17280)	1
  (0, 18290)	1
  (0, 18928)	1
  (0, 19085)	1
  (0, 20906)	1
  (0, 21098)	1
  :	:
  (4, 19876)	1
  (4, 20028)	3
  (4, 20282)	1
  (4, 20452)	1
  (4, 20463)	6
  (4, 20757)	1
  (4, 20955)	1
  (4, 21031)	1
  (4, 21185)	3
  (4, 21295)	1
  (4, 21877)	2
  (4, 22307)	3
  (4, 22712)	2
  (4, 22721)	1
  (4, 23396)	1
  (4, 23550)	1
  (4, 23700)	1
  (4, 24426)	1
  (4, 24860)	1
  (4, 25139)	1
  (4, 25314)	1
  (4, 25544)	1
  (4, 25568)	1
  (4, 25596)	2
  (4, 25755)	1


Word2vec

In [21]:
def word2vec_vectorization(data, embedding_dim=100):

    sentences = [doc.split() for doc in data]

    model = Word2Vec(sentences=sentences, vector_size=embedding_dim, min_count=1)

    word_vectors = []
    for doc in sentences:
        doc_vector = np.zeros(embedding_dim)
        word_count = 0
        for word in doc:
            if word in model.wv:
                doc_vector += model.wv[word]
                word_count += 1
        if word_count > 0:
            doc_vector /= word_count
        word_vectors.append(doc_vector)

    return np.array(word_vectors)

In [22]:
X_word2vec_train = word2vec_vectorization(train_x)
X_word2vec_test = word2vec_vectorization(test_x)
X_word2vec_valid = word2vec_vectorization(valid_x)

In [23]:
print("Word2Vec Vectors for Train Set:")
print(X_word2vec_train[:5])

Word2Vec Vectors for Train Set:
[[-3.95053031e-01  5.08154767e-01 -1.12333063e-01  2.22367156e-01
  -7.38624728e-02 -6.99313405e-01  2.18048980e-01  8.09806332e-01
  -2.94183424e-01 -2.52390337e-01 -6.03771665e-02 -4.36503777e-01
  -8.47052976e-02  1.27807824e-02  2.31503721e-01 -4.10914084e-01
   3.06933855e-01 -6.12914452e-01 -1.31623320e-01 -7.93575567e-01
   2.35608603e-01  2.39857395e-01  3.88205052e-01  9.12706799e-02
  -1.03918553e-01 -1.99783058e-01 -4.27448855e-01 -2.58544528e-01
  -4.37743263e-01  2.19577539e-01  4.76777240e-01  1.72907738e-01
   7.31420122e-01 -2.56820220e-01 -2.33702085e-01  6.98349002e-01
  -2.93073231e-02 -3.04493878e-01 -1.55015800e-01 -8.57888220e-01
   4.83702592e-02 -5.23850879e-01 -8.58108202e-03  9.87060999e-02
   1.74597416e-01 -2.96466462e-01 -4.16451331e-01  9.44628302e-02
   2.23200117e-01  4.25982159e-01  9.91896101e-02 -4.53807141e-01
  -2.59308332e-01  6.63223003e-02 -1.42816023e-01  2.93066483e-01
   2.26372559e-01 -1.80575744e-01 -5.6515630

In [24]:
print("Word2Vec Vectors for Test Set:")
print(X_word2vec_test[:5])

Word2Vec Vectors for Test Set:
[[-2.87274286e-01  2.49482661e-01 -2.24367075e-01  5.76829761e-02
   4.44031896e-01 -4.68382762e-01 -1.96421773e-01  1.18468584e+00
  -3.58482842e-01  3.44219901e-01 -6.22628147e-01 -1.11231984e+00
  -1.79515058e-01 -6.02408996e-02  9.68538926e-02 -5.80462090e-01
   3.15036071e-02 -4.94240317e-01  8.33213076e-02 -9.80036397e-01
   1.45471560e-01  9.43043387e-02 -1.48236780e-01 -4.61829133e-01
   8.79301024e-02 -1.98124991e-01 -6.20733510e-02 -7.53946030e-01
  -4.88460684e-01  4.21770252e-02  6.74041927e-01 -1.14529089e-01
   1.24901171e-01 -3.20679809e-02 -3.53186269e-02  5.90883408e-01
  -1.32416960e-01 -4.44979858e-01 -2.44488663e-01 -8.92504942e-01
  -6.95396519e-02 -5.01648577e-01  1.78899079e-02  2.88677671e-01
   1.97045617e-01 -3.20539348e-01 -7.44403185e-01 -2.56312731e-01
   1.36309076e-01  3.06969031e-01  4.61217665e-01 -3.25233410e-01
  -3.56357618e-01  5.88648472e-02 -4.86510921e-01  2.00516094e-01
   4.74483841e-01  7.94837963e-02 -7.55092877

In [25]:
print("Word2Vec Vectors for Valid Set:")
print(X_word2vec_valid[:5])

Word2Vec Vectors for Valid Set:
[[-3.04271541e-01  4.00495588e-01  1.37405048e-01  1.87451322e-01
   6.15201143e-02 -6.80174524e-01  2.92160929e-01  8.04001654e-01
  -2.36909358e-01 -1.62755731e-01  8.28604870e-02 -6.79561362e-01
  -1.39196027e-01  2.93508268e-01  2.57332224e-03 -3.75919142e-01
   1.68619058e-01 -5.36494431e-01 -1.17747028e-01 -7.55904046e-01
   3.31541319e-01  4.24329665e-01  2.37314641e-01 -7.98181670e-02
  -1.33469770e-01 -1.44298454e-01 -2.74705490e-01 -1.29896597e-01
  -2.23959440e-01 -3.28436732e-02  4.08020494e-01 -2.07847339e-02
   3.35789027e-01 -3.75706105e-01  6.95185338e-02  7.03619945e-01
   8.53409843e-02 -4.16430142e-01 -1.86391676e-01 -6.27422612e-01
   9.67668974e-02 -3.69319834e-01 -1.31053683e-01  6.68195071e-02
   5.16642150e-01 -1.22410056e-01 -2.21457570e-01 -1.37323199e-01
   2.98612626e-01  3.63431016e-01  1.63466900e-01 -3.28475829e-01
  -1.66693943e-01  5.52208951e-02 -2.07127351e-01  2.37575943e-01
   1.52029410e-01 -1.19890726e-01 -5.1278502

Doc2vec

In [26]:
# Doc2Vec Vectorization
def doc2vec_vectorization(data, vector_size=100):

    model = Doc2Vec(data, vector_size=vector_size, window=4, min_count=2, workers=4, epochs=40)

    doc_vectors = [model.dv[i] for i in range(len(data))]

    return np.array(doc_vectors)

In [27]:
documents_train = [TaggedDocument(doc.split(), [i]) for i, doc in enumerate(train_x)]
documents_valid = [TaggedDocument(doc.split(), [i]) for i, doc in enumerate(valid_x)]
documents_test = [TaggedDocument(doc.split(), [i]) for i, doc in enumerate(test_x)]

In [28]:
X_doc2vec_train = doc2vec_vectorization(documents_train)
X_doc2vec_valid = doc2vec_vectorization(documents_valid)
X_doc2vec_test = doc2vec_vectorization(documents_test)

In [29]:
print("Doc2Vec Vectors for Train Set:")
print(X_doc2vec_train[:5])

Doc2Vec Vectors for Train Set:
[[-2.84636766e-01 -2.43592605e-01 -9.62786853e-01  6.17962420e-01
  -6.11012522e-03  8.61689389e-01  6.60824418e-01  3.23107004e-01
  -3.53049159e-01 -1.44031614e-01  6.75355136e-01  1.07075691e+00
  -5.09679258e-01 -3.94865930e-01 -9.39743146e-02 -1.33390635e-01
  -5.19623995e-01 -4.04502928e-01 -3.77485991e-01 -7.55980551e-01
  -4.16810811e-01  9.79826689e-01  7.85190105e-01 -1.18788265e-01
  -6.30341703e-03 -8.74947250e-01 -6.16891861e-01 -9.71440077e-02
  -3.95412922e-01 -2.70454079e-01  7.49114811e-01  1.28467178e+00
   9.11271155e-01 -3.57881308e-01 -9.47162747e-01  5.33240736e-01
  -3.01414818e-01  2.21048236e-01  6.02551937e-01 -1.36711276e+00
   3.22673023e-01 -1.29276407e+00 -5.23169160e-01  2.46956751e-01
  -5.99101894e-02 -4.24059570e-01 -6.31050318e-02 -7.81454802e-01
  -3.21671069e-01 -5.41499853e-01  9.94674325e-01 -3.68379662e-03
  -9.03335512e-01 -2.40741476e-01 -2.68742353e-01 -5.80566227e-01
  -3.85060251e-01 -8.66856128e-02 -1.21408939

In [30]:
print("Doc2Vec Vectors for Test Set:")
print(X_doc2vec_test[:5])

Doc2Vec Vectors for Test Set:
[[-6.83211625e-01  2.68082166e+00 -1.24217141e+00  9.90387857e-01
   1.42812419e+00  8.85792434e-01 -1.06884789e+00  1.15096581e+00
  -1.10066879e+00  3.46930295e-01 -4.73013818e-01 -1.66868329e+00
   1.24584424e+00 -4.72834110e-01 -6.97734952e-01  1.42622709e-01
  -7.95471221e-02 -1.66728228e-01  4.32144016e-01 -4.61739212e-01
  -1.31131399e+00  2.33059689e-01  6.40773237e-01 -1.34323508e-01
   1.87215865e+00 -1.01220809e-01 -1.42181849e+00 -1.29064312e-04
   4.04560030e-01 -6.14537716e-01 -3.36628228e-01  9.82474566e-01
   1.43013462e-01  1.46269536e+00  1.42901897e+00 -2.87411302e-01
   8.02121222e-01 -1.09557426e+00 -8.55430841e-01 -3.88504028e-01
  -3.16451579e-01 -2.63549894e-01  5.95098771e-02 -1.82315755e+00
  -7.47492552e-01 -1.50956821e+00 -3.99835885e-01  3.67211550e-01
  -1.10478319e-01 -5.78768849e-01 -1.19437921e+00 -4.77404101e-03
  -5.12434304e-01  1.26307070e+00  8.17429900e-01  1.29026961e+00
  -2.41088271e-01 -7.80528545e-01 -3.14324647e

In [31]:
print("Doc2Vec Vectors for Valid Set:")
print(X_doc2vec_valid[:5])

Doc2Vec Vectors for Valid Set:
[[-0.29580098  0.19801891  0.3422904   0.02389556 -0.03582233 -0.45970502
  -0.37429804  0.58625084 -0.50101936 -0.13919537 -0.4112408  -0.18267308
   0.07322112  0.06141551  0.03604908 -0.32647175 -0.06577857  0.09593156
   0.0674703  -0.627942   -0.04861861 -0.04599873 -0.3837731  -0.28995758
  -0.33828658  0.09035003 -0.16205947 -0.04414544  0.20229699  0.5743874
  -0.3756769   0.12517379  0.10049552  0.25395703  0.1521422   0.7521937
   0.5920995  -0.19460678 -0.19766821  0.00357246 -0.15130427 -0.07453191
  -0.15726288 -0.20214656  0.16700777 -0.29391867 -0.48869443 -0.1995847
  -0.25409183  0.4917995   0.04268108 -0.33992642  0.05799425  0.3415116
  -0.30585423  0.47686356  0.3123979  -0.34572557 -0.08829331  0.579092
   0.41175452  0.15135887 -0.3528502  -0.06951641 -0.43304294  0.33486953
  -0.03558158  0.694304   -0.13241903  0.2457236  -0.17783944 -0.19115572
   0.19137321 -0.05314714  0.16397862  0.45187035  0.10608822  0.35579827
  -0.15866087

In [32]:
print("TF-IDF Train Matrix Shape:", xtrain_tfidf.shape)
print("TF-IDF Test Matrix Shape:", xtest_tfidf.shape)
print("TF-IDF Valid Matrix Shape:", xvalid_tfidf.shape)

TF-IDF Train Matrix Shape: (4508, 5000)
TF-IDF Test Matrix Shape: (2242, 5000)
TF-IDF Valid Matrix Shape: (1128, 5000)


In [33]:
print("Count Vectorization Train Matrix Shape:", xtrain_count.shape)
print("Count Vectorization Test Matrix Shape:", xtest_count.shape)
print("Count Vectorization Valid Matrix Shape:", xvalid_count.shape)

Count Vectorization Train Matrix Shape: (4508, 26037)
Count Vectorization Test Matrix Shape: (2242, 26037)
Count Vectorization Valid Matrix Shape: (1128, 26037)


In [34]:
print("Word2Vec Train Matrix Shape:", X_word2vec_train.shape)
print("Word2Vec Test Matrix Shape:", X_word2vec_test.shape)
print("Word2Vec Valid Matrix Shape:", X_word2vec_valid.shape)

Word2Vec Train Matrix Shape: (4508, 100)
Word2Vec Test Matrix Shape: (2242, 100)
Word2Vec Valid Matrix Shape: (1128, 100)


In [35]:
print("Doc2Vec Train Matrix Shape:", X_doc2vec_train.shape)
print("Doc2Vec Test Matrix Shape:", X_doc2vec_test.shape)
print("Doc2Vec Valid Matrix Shape:", X_doc2vec_valid.shape)

Doc2Vec Train Matrix Shape: (4508, 100)
Doc2Vec Test Matrix Shape: (2242, 100)
Doc2Vec Valid Matrix Shape: (1128, 100)


In [36]:
print("Doc2Vec Vectors for Training Set:")
print(X_doc2vec_train[:5])

Doc2Vec Vectors for Training Set:
[[-2.84636766e-01 -2.43592605e-01 -9.62786853e-01  6.17962420e-01
  -6.11012522e-03  8.61689389e-01  6.60824418e-01  3.23107004e-01
  -3.53049159e-01 -1.44031614e-01  6.75355136e-01  1.07075691e+00
  -5.09679258e-01 -3.94865930e-01 -9.39743146e-02 -1.33390635e-01
  -5.19623995e-01 -4.04502928e-01 -3.77485991e-01 -7.55980551e-01
  -4.16810811e-01  9.79826689e-01  7.85190105e-01 -1.18788265e-01
  -6.30341703e-03 -8.74947250e-01 -6.16891861e-01 -9.71440077e-02
  -3.95412922e-01 -2.70454079e-01  7.49114811e-01  1.28467178e+00
   9.11271155e-01 -3.57881308e-01 -9.47162747e-01  5.33240736e-01
  -3.01414818e-01  2.21048236e-01  6.02551937e-01 -1.36711276e+00
   3.22673023e-01 -1.29276407e+00 -5.23169160e-01  2.46956751e-01
  -5.99101894e-02 -4.24059570e-01 -6.31050318e-02 -7.81454802e-01
  -3.21671069e-01 -5.41499853e-01  9.94674325e-01 -3.68379662e-03
  -9.03335512e-01 -2.40741476e-01 -2.68742353e-01 -5.80566227e-01
  -3.85060251e-01 -8.66856128e-02 -1.21408